In [14]:
import os, requests
import matplotlib.pyplot as plt
import nglview as nv
from ipywidgets import Output, HBox
import pandas as pd

# Delete log files and results
for file in os.listdir():
    if not file.endswith(('ipynb','png','pdb','xtc')):
        print("rm", file)
        os.remove(file)

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **MD_id**: PDB code of the protein structure (e.g. [A01M6](https://mmb-dev.mddbr.eu/#/id/A01M6/overview))

In [3]:
MD_id = 'A01M6'
steps = 1500

<a id="fetch"></a>
***
## Fetching structure and trajectory

In [4]:
# Check if PDB and XTC files already exist
if not (os.path.exists(f"{MD_id}.pdb") and os.path.exists(f"{MD_id}.xtc")):
    # Download PDB
    headers = {'accept': 'chemical/x-pdb'}
    url = f"https://mmb-dev.mddbr.eu/api/rest/v1/projects/{MD_id}/files/structure.pdb"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        pdb_content = response.text
        # Save the PDB content to a file
        with open(f"{MD_id}.pdb", 'w') as pdb_file:   
            pdb_file.write(pdb_content)
        print(f"Saved PDB file")
    else:
        print(f"Error for the PDB file: {response.status_code}")
        
    # Download XTC
    headers2 = {'accept': 'application/octet-stream'}
    url2= f'https://mmb-dev.mddbr.eu/api/rest/v1/projects/{MD_id}/files/trajectory.xtc'
    response2 = requests.get(url2, headers=headers2)
    if response2.status_code == 200:
        xtc_content = response2.content
        with open(f"{MD_id}.xtc", 'wb') as xtc_file:
            xtc_file.write(xtc_content)
        print(f"Saved XTC file")
    else:
        print(f"Error for the XTC file: {response2.status_code}")

In [5]:
# common arguments for all the components
input_top_path = f'{MD_id}.pdb'
input_traj_path = f'{MD_id}.xtc'

<a id="density"></a>
***
## Density profile

Calculate density...<br>
***
**Building Blocks** used:
 - [CpptrajDensity](https://biobb-mem.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_density) from **biobb_mem.ambertools.cpptraj_density**
***

In [6]:
from biobb_mem.ambertools.cpptraj_density import cpptraj_density

prop = {
    'mask': '::M ::A,B',
    'steps': steps,
}
cpptraj_density(input_top_path=input_top_path,
            input_traj_path=input_traj_path,
            output_cpptraj_path='density_mask.dat',
            properties=prop)

2024-11-26 16:29:17,257 [MainThread  ] [INFO ]  Module: biobb_mem.ambertools.cpptraj_density Version: 5.0.2
2024-11-26 16:29:17,258 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_4a5f5914-4a33-4d02-b571-075c4e5da0af directory successfully created
2024-11-26 16:29:17,260 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_4a5f5914-4a33-4d02-b571-075c4e5da0af
2024-11-26 16:29:17,862 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_4a5f5914-4a33-4d02-b571-075c4e5da0af
2024-11-26 16:29:17,863 [MainThread  ] [INFO ]  cpptraj -i e5d1e2eb-82e3-4e3d-a932-e0928ce6adc9/instructions.in

2024-11-26 16:29:18,036 [MainThread  ] [INFO ]  Executing: cpptraj -i e5d1e2eb-82e3-4e3d-a932-e0928ce6adc9/instructions.in...
2024-11-26 16:29:18,036 [MainThread  ] [INFO ]  Exit code: 0
2024-11-26 16:29:18,037 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V6.18.1 (AmberTools)
    ___  

0

### Visualization

In [7]:
def plot_and_view(den,view,labels,axis='z'):
    axis = f'#{axis.upper()}'
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    cls = den.columns[1:]
    cls = den.columns[1:]
    for i,cl in enumerate(cls):
        if i%2==0:
            ax.plot(den[axis], den[cl], label=labels[i//2])
            cl0 = cl
        else:
            ax.fill_between(den[axis], den[cl0] - den[cl], den[cl0] + den[cl],label='SD',color='red')
    ax.set_title('Mass Density');ax.set_xlabel('Å')
    ax.set_ylabel('Density'); ax.legend()

    # Put plot in a widget and display both widgets in a row
    out = Output()
    with out: plt.show()
    box = HBox([out,view]); display(box)
    view._remote_call('setSize', target='Widget', args=['100%', '100%'])
    view.control.spin([0, 1, 0], -1.57) 

In [8]:
# Mask
view = nv.show_file(input_top_path)
den = pd.read_csv(f'density_mask.dat', sep='\s+')
plot_and_view(den,view,['Membrane','Protein'])
view.update_ball_and_stick(color='blue')
view.update_cartoon(color='orange')

<a id="leaflets"></a>
***
## Leafleats assignation

Lyphilic is faster that fatslim, it algorithm is simpler, assumes a plane membrane, it let you do the assignation of leafleats for every frame and use this information for analysis like flip-flop <br>
***
**Building Blocks** used:
 - [LPPAssignLeaflets](https://biobb-mem.readthedocs.io/en/latest/lipyphilic_biobb.html) from **from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets**
***

![DPPC](DPPC.png "DPPC")

In [9]:
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import lpp_assign_leaflets

prop = {
    'lipid_sel': '(resname DPPC and name P8)',
    'steps': steps
}
lpp_assign_leaflets(input_top_path=input_top_path,
                    input_traj_path=input_traj_path,
                    output_leaflets_path= 'leaflets_data.csv',
                    properties=prop)

/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


2024-11-26 16:29:20,080 [MainThread  ] [INFO ]  Module: biobb_mem.lipyphilic_biobb.lpp_assign_leaflets Version: 5.0.2
2024-11-26 16:29:20,080 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_730860c8-467e-4ee8-a3bb-a95d9a08cd57 directory successfully created
2024-11-26 16:29:20,082 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_730860c8-467e-4ee8-a3bb-a95d9a08cd57
2024-11-26 16:29:20,642 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_730860c8-467e-4ee8-a3bb-a95d9a08cd57
2024-11-26 16:29:20,901 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_730860c8-467e-4ee8-a3bb-a95d9a08cd57']
2024-11-26 16:29:20,901 [MainThread  ] [INFO ]  


0

### Visualization

In [10]:
df = pd.read_csv('leaflets_data.csv')
view = nv.show_file(input_top_path)
top_idx = df[(df['frame'] == 0) & (df['leaflet_index'] == 1)]['resindex'].values
bot_idx = df[(df['frame'] == 0) & (df['leaflet_index'] == -1)]['resindex'].values

view.update_ball_and_stick(selection='all', opacity=0.0)   # delete membrane
view.add_ball_and_stick(selection=", ".join(map(str, top_idx)), color='blue')
view.add_ball_and_stick(selection=", ".join(map(str, bot_idx)), color='yellow')
view.control.spin([0, 1, 0], -1.57)
view

NGLWidget()

<a id="channel"></a>
***
## Channel

Chanel... <br>
***
**Building Blocks** used:
 - [MDAHole](https://biobb-mem.readthedocs.io/en/latest/mdanalysis_biobb.html#mdanalysis_biobb.mda_hole.MDAHole) from **from biobb_mem.mdanalysis_biobb.mda_hole**
***

In [11]:
from biobb_mem.mdanalysis_biobb.mda_hole import mda_hole

prop = {
    'select': 'protein',
    'steps': steps,
    'sample': 0.1
}
mda_hole(input_top_path=input_top_path,
         input_traj_path=input_traj_path,
         output_hole_path='hole.vmd',
         properties=prop)

2024-11-26 16:29:21,449 [MainThread  ] [INFO ]  Module: biobb_mem.mdanalysis_biobb.mda_hole Version: 5.0.2
2024-11-26 16:29:21,449 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_74cc1417-8ff2-4ce0-bd6a-3ee1641670f6 directory successfully created
2024-11-26 16:29:21,452 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_74cc1417-8ff2-4ce0-bd6a-3ee1641670f6
2024-11-26 16:29:22,029 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_74cc1417-8ff2-4ce0-bd6a-3ee1641670f6


/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/MDAnalysis/analysis/hole2/hole.py:495: DeprecationWarning: This class has been moved to the MDAKit `hole2-mdakit` (https://github.com/MDAnalysis/hole2-mdakit) and will be removed in version 3.0.
  warnings.warn(wmsg, DeprecationWarning)
/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/MDAnalysis/lib/util.py:567: DeprecationWarning: This method is deprecated as of MDAnalysis version 2.7.0 and will be removed in version 3.0.0. Please use shutil.which instead.
  warnings.warn(wmsg, DeprecationWarning)


2024-11-26 16:29:35,203 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/sandbox_74cc1417-8ff2-4ce0-bd6a-3ee1641670f6']
2024-11-26 16:29:35,204 [MainThread  ] [INFO ]  


0

In [ ]:
!conda install -c conda-forge vmd -y # si se instala en el entorno conda da problemas

In [12]:
# TODO: cargar más frame (lento pero se resolvería con H5MD) y mirar por que algunos caneles salen torcidos
vmd_script = f"""
mol addfile {input_traj_path} type xtc first 0 last 1 step {steps} waitfor all
animate delete beg 0 end 0
source hole.vmd
"""

# Join the contents and write to a new file
with open('vmd_script.vmd', 'w') as fl:
    fl.write(vmd_script)

!vmd -pdb {input_top_path} -e vmd_script.vmd

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 16 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 55GB (88%)
Info) No CUDA accelerator devices available.
Info) OpenGL renderer: Mesa Intel(R) UHD Graphics 630 (CML GT2)
Info)   Features: STENCIL MSAA(4) MDE CVA MTX NPOT PP PS GLSL(OVFS) 
Info)   Full GLSL rendering mode is available.
Info)   Textures: 2-D (16384x16384), 3-D (512x512x512), Multitexture (8)
Info) Detected 8 XInput devices, 1 usable device:
Info)   [8] xwayland-pointer-gestures:16, type: 242